In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import requests
import json
import tweepy

In [2]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAI0UcAEAAAAAGv32GH%2B3%2ByYoz0Els5ZBPcq%2FrDc%3Dezec1hn1fdogEvTg5DER5nHA6245j6UvAUDC8x5qn9YaUAR2bY"
ACCESS_TOKEN = "1519935617290833920-G7EiFZGJsIOdI7KTezC7oYFSN4ppdq"
ACCESS_TOKEN_SECRET = "Vwg4tWhX3VPQvQkqQbIJfjq7xffBsskPtSo5qYVTXwKZU"
API_KEY = "IaLlNLbiKRSygOrb1Hc211mCW"
API_KEY_SECRET = "kKlFyX8p7kC88OZUE8NUqslFtfNnui2uRCJpzi0S0B5al5fs2w"

In [3]:
authenticate = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
authenticate.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [6]:
api = tweepy.API(authenticate, wait_on_rate_limit=True)
res = api.search_full_archive(query='gas price', label='gas')
res

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve